### Serving models on the cloud (Azure)

https://learn.microsoft.com/en-us/azure/machine-learning/concept-workspace?view=azureml-api-2

### TODO Recording

- Login to portal.azure.com
- Click through to the paid subscription and copy the subscription ID
- Create a new resource group and call it loony-mlflow-rg in the East US region
- Search for Azure Machine Learning and select that service
- Show that no workspaces have been created
- Create a new workspace here called "loony_test", same resource group, location East US
- Once the workspace is created go to Resource Groups on the portal
- Show that a RG with the prefix "Default" has been created for the East US region
- This RG needs to be there for new workspaces to be able to be created programmatically

### Notes
- This is to get around a bug when we programmatically create a WS, this fails unless we've already created a workspace in our account earlier (some set up does not happen if we only create a WS programmatically)

For serving the logged model in Azure ML studio, we need to install some packages related to azure

In [ ]:
pip install azureml-core

In [ ]:
pip install azure-ai-ml azure-identity

In [ ]:
pip install azureml-mlflow

In [ ]:
from azureml.core import Workspace

ws = Workspace.create(
    name = 'loony-mlflow-wsp',
    subscription_id = '<subscription_id>',
    resource_group = 'loony-mlflow-rg',
    create_resource_group = False,
    location = 'eastus'
)

### TODO Recording:

- Go to the Azure portal
- In Azure Machine Learning show that the loony-mlflow-wsp has been created
- Click through to the workspace
- Launch Azure ML Studio

##### MLflow UI
- Now go to the MLflow UI
- Go to Experiments -> "random forest model default parameters" run
- In the artifacts you should be able to see the full path of where the model is deployed
- Click on copy so you have that
- We will need this path in the cell below
- We only need the path that starts with '/Users' remove the file:// prefix

In [ ]:
from azureml.core.model import Model

model = Model.register(
    workspace = ws, 
    model_path = '<model_path>', 
    model_name = 'churn-prediction-model'
)

### TODO Recording

- After registering the model go to the Azure Portal
- In Azure ML Studio click on "Models" on the left hand side
- Show the model registered there
- Click through to the model and show details
- From the top tabs click on Versions
- Click on Artifacts and select the model/ folder and show that the MLflow artifacts are here
- Click on Endpoints and there should be no endpoints here

In [ ]:
import json
from mlflow.deployments import get_deploy_client

# Create the deployment configuration.
# If no deployment configuration is provided, 
# then the deployment will happen using ACI (Azure Container Instance).
deploy_config = {"computeType": "aci"}

# Write the deployment configuration into a file.
deployment_config_path = "deployment_config.json"

with open(deployment_config_path, "w") as outfile:
    outfile.write(json.dumps(deploy_config))

### TODO Recording:

- Go to the Azure Portal to the loony-mlflow-wsp
- At the top right there will be an MLflow tracking URI
- Copy that over and paste it into the cell below

In [ ]:
# Set the tracking uri in the deployment client.
client = get_deploy_client("<tracking_url>")

# MLflow requires the deployment configuration to be passed as a dictionary.
config = {"deploy-config-file": deployment_config_path}
model_name = 'churn-prediction-model'
model_version = 1

In [ ]:
mlflow.set_tracking_uri('<tracking_uri>')

define the model path and the name is the service name
if model is not registered, it gets registered automatically and a 
name is autogenerated using the "name" parameter below

In [ ]:
client.create_deployment(
    model_uri = f"models:/{model_name}/{model_version}",
    config = config,
    name = "churn-pred-model-aci-deployment",
)

### TODO Recording

- Go to Azure Portal
- Click on Endpoints at the left hand side
- Show that there is one endpoint there
- Click through and show the the endpoint is in the process of being created
- Stay on the Azure portal till the end point is Healthy - will take 20-25 minutes
- Copy over the REST endpoint (will need it in the cell below)

End point is consumed for getting prediction

In [ ]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and \
        not os.environ.get('PYTHONHTTPSVERIFY', '') and \
        getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

# this line is needed if you use self-signed certificate in your scoring service.
allowSelfSignedHttps(True)

# Request data goes here
data =  {
  "input_data": {
    "columns": [
      "Age",
      "FrequentFlyer",
      "AnnualIncomeClass",
      "ServicesOpted",
      "AccountSyncedToSocialMedia",
      "BookedHotelOrNot"
    ],
    "index": [6],
    "data": [[28, 'Yes', 2, 6, 'No', 'Yes']]
  }
}

# data =  {
#   "input_data": {
#     "columns": [
#       "Age",
#       "FrequentFlyer",
#       "AnnualIncomeClass",
#       "ServicesOpted",
#       "AccountSyncedToSocialMedia",
#       "BookedHotelOrNot"
#     ],
#     "index": [8],
#     "data": [[23, 'No', 0, 1, 'No', 'No']]
#   }
# }


body = str.encode(json.dumps(data))

url = '<REST_endpoint>'

headers = {'Content-Type':'application/json'}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, 
    # which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

### TODO Recording

- Run the cell above with the first data - should get 1
- Comment out the first data and uncomment the second
- Run the cell - should get 0

Deleting the registered model

### TODO Recording:

- Go to the MLflow UI
- Go to models
- Go to the Archived version and delete that
- Go to the Production version Archive then delete
- Delete the churn_prediction_model
- Go to Experiments and delete the Experiment

#### Azure portal
- Go to the Azure Portal
- Home -> loony-mlflow-wsp
- Delete the workspace